## Event Sourcing with Amazon EventBridge

This example demonstrates using Amazon EventBridge to invoke Lambda functions when events are added to the EventBus. In the example event rules are used to trigger the Lambda functions. There is an event rule created for new orders and a rule that catches update operations. One Lambda function inserts into a DynamoDB table while the other function updates the item in the table.

![architecture](../images/architecture_1.png "Architecture")

**Jupyter Notebook Scripts**

1. The first script sends new orders to the gateway endpoint. The event rule for new orders filters on the ``` 'eventtype'::['new_order']``` attribute.


2. The second script sends an order update to the gateway endpoint. The event rule for order updates filters on the ```'eventtype':['update_order']``` attribute.


3. Before you run the final script you need to create an event rule to catch delete operations by using the ```'eventtype':['delete_order']``` attribute, the delete script includes the ```'eventtype':['delete_order']```.


**Note:** Make sure you set gwid to your gateway id using - gwid = '...'

In [1]:
import boto3, requests, datetime
from random import randrange

#Set gateway id
gwid = 'bc6xmpkpqd'

city_list=["Adelaide","Brisbane","Canberra","Darwin","Geelong","Gold Coast","Hobart","Melbourne","Perth","Sydney","Wollongong"]
coffeetype_list=["Short Black","Flat White","Latte","Long Black"]
coffeesize_list=[{"size":"Small","price":"3.5"},{"size":"Medium","price":"4.0"},{"size":"Large","price":"4.5"},{"size":"x-Large","price":"5.0"}]

url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com')

for i in range(100):
    accountid = 'a' + str(i)
    vendorid = 'v' + str(i)
    orderdate = str(datetime.datetime.now())
    
    #Original value, random selection
    coffeesize = randrange(4)
    #-------------------------

    quantity = randrange(10)+1
    city = city_list[randrange(11)]
    eventtype="new_order"
    response = requests.post(url,json={'order':{
            'accountid': accountid,
            'orderdate':orderdate,
            'vendorid': vendorid,
            'city':city,
            'details':{
                'coffeetype': coffeetype_list[randrange(4)],
                'coffeesize': coffeesize_list[coffeesize]["size"],
                'unitprice': coffeesize_list[coffeesize]["price"],
                'quantity': quantity
            },
            'eventtype':[eventtype]
        }
    })

print(response)

<Response [200]>


**Update** items in DynamoDB.


After executing the update script, check DynamoDB for the following changes made to the first 10 records.

* coffeetype is changed to Flat White 

* coffeesize is changed to Large

In [2]:
import json, boto3, requests, datetime

#Set gateway id   
city_list=["Adelaide","Brisbane","canberra","Darwin","Geelong","Hobart","Melbourne","Perth","Sydney","Wollongong"]

url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com')

for i in range(10):
    accountid = 'a' + str(i)
    vendorid = 'v' + str(i)
    orderdate = str(datetime.datetime.now())
    
    #Changes-----------------
    coffeetype = 'Flat white'
    coffeesize = 'Large'
    #------------------------
    
    unitprice = "4.50"
    quantity = 1
    city = city_list[i]
    
    #EventBus rule filters on event type
    eventtype="update_order"
    
    response = requests.post(url,json={'order':{
            'accountid': accountid,
            'orderdate':orderdate,
            'vendorid': vendorid,
            'city':city,
            'details':{
                'coffeetype': coffeetype,
                'coffeesize': coffeesize,
                'unitprice': unitprice,
                'quantity': quantity
            },
            'eventtype':[eventtype]
        }
    })

print(response)


<Response [200]>


**Amazon EventBridge**. Using Amazon EventBridge create a rule that catches delete operations using the following settings.

Event Bus: ```event_sourcing_eventbridge```

Name: ``` event_sourcing_eventbridge_delete_order```

Target: ```Lambda```

Function: ```event_sourcing_eventbridge_delete_item```

Event pattern: 
```
{
  "detail": {
    "order": {
      "eventtype": ["delete_order"]
    }
  }
}
```


**Delete Item**. After executing the script check DynamoDB for the an item with accountid':'a0' and 'vendorid':'v0'

In [ ]:
delete_url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com')

#EventBus rule filters on event type
eventtype="delete_order"
    
response = requests.post(delete_url,json={'order':{'accountid':'a0','vendorid':'v0','eventtype':[eventtype]}})

print(response)

**Clean up**. You will need delete the ```event_sourcing_eventbridge_delete_order``` before you can destroy the stack. This can be done manually or you can run the following AWS Cli commands.

```
aws events list-targets-by-rule  --event-bus-name event_sourcing_eventbridge --rule "event_sourcing_eventbridge_delete_order" --query 'Targets[0].[Id][0]'

aws events remove-targets --event-bus-name event_sourcing_eventbridge --rule "event_sourcing_eventbridge_delete_order" --ids "<returned form last query>"

aws events delete-rule --event-bus-name event_sourcing_eventbridge --name "event_sourcing_eventbridge_delete_order"
```